In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
CHP_ID = "5425"
SUBMODEL = "cond-detr-50" # "cond-detr-50"
MODEL_PATH = f"logs/{SUBMODEL}/checkpoint-{CHP_ID}"
# IMAGE_SHAPE = 1333 _Shape{IMAGE_SHAPE}
THR = 0.05
iou_threshold = 0.8
FILE_NAME = f"{SUBMODEL}_THR{THR*100:.3f}_IOU{iou_threshold:.3f}_ID{CHP_ID}_P"

In [3]:
from transformers import (
	AutoImageProcessor,
	AutoModelForObjectDetection,
	ConditionalDetrImageProcessor,
    ConditionalDetrForObjectDetection
)
from PIL import Image
import torch
from torchvision.ops import nms

import pandas as pd
import numpy as np

In [4]:
from zindi_code.dataset import load_and_format
from zindi_code import CLS_MAPPER

image_folder = "zindi_data/images"

test = pd.read_csv("zindi_data/Test.csv")
test.sample(5)

/data/home/eak/learning/zindi_challenge/lacuna_chl/.venv/lib/python3.9/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.16 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


,Image_ID,class,confidence,ymin,xmin,ymax,xmax
990,id_gmkmdd7pqc.jpg,NaN,NaN,NaN,NaN,NaN,NaN
899,id_1mf35d73se.jpg,NaN,NaN,NaN,NaN,NaN,NaN
1069,id_u9f214ojg0.jpg,NaN,NaN,NaN,NaN,NaN,NaN
1038,id_1csdphjt28.jpg,NaN,NaN,NaN,NaN,NaN,NaN
4,id_tirid4qt68.jpg,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
model_pth = MODEL_PATH

image_processor: ConditionalDetrImageProcessor = AutoImageProcessor.from_pretrained(
    model_pth
)
model: ConditionalDetrForObjectDetection = AutoModelForObjectDetection.from_pretrained(
    model_pth
).to(
    "cuda"
)  # .train(False)

In [6]:
image_processor

ConditionalDetrImageProcessor {
  "do_convert_annotations": true,
  "do_normalize": true,
  "do_pad": true,
  "do_rescale": true,
  "do_resize": true,
  "format": "coco_detection",
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_processor_type": "ConditionalDetrImageProcessor",
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "pad_size": null,
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "longest_edge": 1333,
    "shortest_edge": 800
  }
}

In [7]:
model.config.id2label

{0: 'Trophozoite', 1: 'WBC'}

In [8]:
@torch.no_grad()
def make_predictions(images: list[Image.Image]):
	inputs = image_processor(images=images, return_tensors="pt").to("cuda")
	outputs = model(**inputs)
	target_sizes = torch.tensor([image.size[::-1] for image in images])
	return image_processor.post_process_object_detection(
		outputs, threshold=THR, target_sizes=target_sizes
	)

def load_transform(path: str):
	return Image.open(os.path.join(image_folder, path)).convert("RGB")
	image = Image.open(os.path.join(image_folder, path))
	return np.array(image.convert("RGB"))[:, :, ::-1]

def load_images(image_pths: list[str]):
	return [
		load_transform(image_pth)
		for image_pth in image_pths
	]


def predicts(image_pths: list[str]):
	images = load_images(image_pths)
	results = make_predictions(images)
	predictions = []
	for image_pth, result in zip(image_pths, results):
		prediction = []
		if len(result["boxes"]):
			indices = nms(result["boxes"], result["scores"], iou_threshold)
			if not len(indices):
				continue
			for score, label, box in zip(
				result["scores"][indices],
				result["labels"][indices],
				result["boxes"][indices],
			):
				x1, y1, x2, y2 = (round(i, 2) for i in box.tolist())
				prediction.append(
					[
						image_pth,
						model.config.id2label[label.item()],
						round(score.item(), 3),
						x1,
						y1,
						x2,
						y2
					]
				)
		if not len(prediction):
			prediction.append([image_pth] + ["NEG", 1., 0, 0, 0, 0,])
		predictions.extend(prediction)
	return pd.DataFrame(
		predictions, columns=["Image_ID", "class", "confidence", "xmin", "ymin", "xmax", "ymax"]
	)

In [9]:
from tqdm import tqdm

In [11]:
batch_size = 16
test_images = test["Image_ID"].unique()
results = [
	predicts(test_images[i : i + batch_size])
	for i in tqdm(
		range(0, len(test_images), batch_size), total=len(test_images) // batch_size + 1
	) if i < len(test_images)
]

100%|██████████| 74/74 [04:53<00:00,  3.97s/it]


In [12]:
predictions = pd.concat(results, ignore_index=True)

In [13]:
predictions.sample(10)

,Image_ID,class,confidence,xmin,ymin,xmax,ymax
3617,id_zzeqlbyhta.jpg,Trophozoite,0.098,1042.00,526.88,1076.02,557.36
6381,id_q624dnvhoj.jpg,Trophozoite,0.324,508.18,60.01,542.88,90.43
29776,id_35zvz4r71s.jpg,Trophozoite,0.127,663.02,504.75,691.76,530.20
15614,id_p0399v663a.jpg,WBC,0.781,3611.38,1301.37,3791.46,1515.30
32811,id_70s6c8isgd.jpg,Trophozoite,0.058,753.94,959.78,835.03,1041.75
25579,id_rvpk6a4kf5.jpg,Trophozoite,0.057,941.66,534.70,979.83,563.04
30333,id_ck2skvxqtf.jpg,Trophozoite,0.067,1209.67,541.66,1237.58,566.09
32993,id_serc1thxju.jpg,Trophozoite,0.110,3072.44,2009.33,3151.18,2104.26
19863,id_2kulbx4oet.jpg,Trophozoite,0.125,608.08,190.91,639.80,219.24
10146,id_m5expi6ak1.jpg,WBC,0.651,1393.51,744.93,1471.89,811.19


In [14]:
predictions["class"].value_counts(True)

class
Trophozoite    0.868711
WBC            0.126167
NEG            0.005122
Name: proportion, dtype: float64

In [15]:
predictions.to_csv(f"zindi_data/test/prediction_{FILE_NAME}.csv", index=False)

In [16]:
f"zindi_data/test/prediction_{FILE_NAME}.csv"

'zindi_data/test/prediction_cond-detr-50_THR5.000_IOU0.800_ID5425_P.csv'